# Класифікатор спаму: підготовка корпусу SpamAssassin

Відкрита база листів [SpamAssassin Public Corpus](https://spamassassin.apache.org/publiccorpus/) складається з кількох наборів текстових файлів, де кожен файл — дамп email-протоколу для одного листа. Тому, початкова мета — перетворити ці дані в формат, легший для моделювання.

Ми виконаємо такі операції:

1. Залишимо тільки тіла листів (email body), і вилучимо всі службові заголовки email-протоколів.
2. Приберемо всі шумові слова (stop words) — артиклі, прийменники тощо.
3. Замінимо всі URL, числа та email-адреси на службові слова `HTTPADDRESS`, `NUMBER`, `EMAILADDRESS`.
4. Всі слова, що залишилися, нормалізуємо за допомогою [стемінгу (stemming)](https://en.wikipedia.org/wiki/Stemming).
5. Збережемо слова кожного листа у вигляді одного великого JSON-масиву.

In [1]:
import email
import glob
import itertools
import json
import os
import re

In [2]:
import numpy as np
from tqdm import tqdm_notebook as progressbar

Завантажимо необхідні мовні моделі та словники:

In [3]:
import nltk

nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("english")
stemmer = nltk.stem.snowball.EnglishStemmer()

[nltk_data] Downloading package stopwords to C:\Users\Yuriy
[nltk_data]     Guts\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Розділення листів на слова

Відокремлюємо тіла листів на ділимо їх на слова.

In [4]:
re_email_filename = re.compile(r"[0-9a-f]{5}\.[0-9a-f]{32}", re.UNICODE)

In [5]:
re_url = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", flags=re.MULTILINE | re.UNICODE)
re_email = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", flags=re.MULTILINE | re.UNICODE)
re_number = re.compile(r"\b\d+\b", flags=re.MULTILINE | re.UNICODE)
re_hash = re.compile(r"[0-9a-f]{32}", flags=re.MULTILINE | re.UNICODE)
re_non_word = re.compile(r"\W+", flags=re.MULTILINE | re.UNICODE)

In [6]:
def email_body_to_words(body, stopwords, stemmer):
    # Convert to lowercase.
    body = body.lower()

    # Replace URLs.
    body = re_url.sub("HTTPADDRESS", body)

    # Replace email addresses.
    body = re_email.sub("EMAILADDRESS", body)

    # Replace hashes.
    body = re_hash.sub("HASH", body)
    
    # Replace numbers.
    body = re_number.sub("NUMBER", body)

    # Remove non-word characters (punctuation, spacers, etc.)
    body = re_non_word.sub(" ", body)

    # Remove trailing spaces.
    body = body.strip()

    # Remove stopwords and stem the remaining words.
    words = [stemmer.stem(word) for word in body.split() if word not in stopwords]
    
    # Remove technical-looking words (base64, underscores, etc.) words.
    words = [word for word in words if len(word) <= 20 and not "_" in word and not any(ch.isdigit() for ch in word)]
    
    return words

In [7]:
def tokenize_emails_in_folders(folders, filename_regex, stopwords, stemmer):
    emails = []
    
    # Iterate through every folder in our dataset.
    for folder in progressbar(folders, desc="Folders"):
        email_filenames = [
            filename
            for filename in sorted(os.listdir(folder))
            if filename_regex.match(os.path.basename(filename))
        ]

        # Each email is kept as a single file, iterate through them.
        for file in progressbar(email_filenames, desc=os.path.basename(folder)):
            email_path = os.path.join(folder, file)
            
            with open(email_path, "r", encoding="utf-8", errors="ignore") as email_file:
                msg = email.message_from_file(email_file)
                if not msg.is_multipart():
                    body = msg.get_payload()
                    words = email_body_to_words(body, stopwords, stemmer)
                    emails.append(words)
    
    return emails

In [8]:
emails_tokenized_ham = tokenize_emails_in_folders(
    [
        "data/spamassassin/easy_ham",
        "data/spamassassin/easy_ham_2",
        "data/spamassassin/hard_ham"
    ],
    re_email_filename,
    stopwords,
    stemmer
)

In [9]:
emails_tokenized_spam = tokenize_emails_in_folders(
    [
        "data/spamassassin/spam",
        "data/spamassassin/spam_2"
    ],
    re_email_filename,
    stopwords,
    stemmer
)

In [10]:
print("# Ham emails:  ", len(emails_tokenized_ham))
print("# Spam emails: ", len(emails_tokenized_spam))

# Ham emails:   3952
# Spam emails:  1590


In [11]:
def save_as_json_file(obj, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(obj, f)

In [12]:
save_as_json_file(emails_tokenized_ham, "emails-tokenized-ham.json")
save_as_json_file(emails_tokenized_spam, "emails-tokenized-spam.json")

## Побудова словника

Наша мета — присвоїти порядковий номер кожному унікальному слову в базі.

Отримуємо множину всіх унікальних слів:

In [13]:
vocab_set = set(itertools.chain(*emails_tokenized_ham)).union(set(itertools.chain(*emails_tokenized_spam)))

Формуємо з неї словник: _cлово_ -> _номер_.

In [14]:
vocab = {
    word: index
    for index, word in enumerate(sorted(list(vocab_set)))
}

In [15]:
print("Vocabulary length:", len(vocab))

Vocabulary length: 34133


Перевіримо наш словних на кількох випадкових словах.

In [16]:
for word in ["buy", "watch", "discount"]:
    print(word.ljust(10), vocab[word])

buy        3953
watch      32213
discount   7616


Збережемо словник у файл — він нам стане у пригоді надалі.

In [17]:
save_as_json_file(vocab, "vocab.json")

## Огляд результатів

Перевіримо, як наш метод очищення листів працює на прикладі довільного листа.

In [18]:
sample_email = email.message_from_string(open("data/spamassassin/easy_ham/01306.01273f7d32eaabde7b20f220e13eb927").read())

In [19]:
sample_body = sample_email.get_payload()
print(sample_body)

Hello,

Has anyone made a working source RPM for dvd::rip for Red Hat 8.0?
Matthias has a spec file on the site for 0.46, and there are a couple of
spec files lying around on the dvd::rip website, including one I patched
a while ago, but it appears that the Makefile automatically generated is
trying to install the Perl libraries into the system's, and also at the
moment dvd::rip needs to be called with PERLIO=stdio as it seems to not
work with PerlIO on RH8's Perl.

Not too sure what the cleanest way to fix this is - anyone working on
this?

Thanks,

-- 
MichÃ¨l Alexandre Salim
Web:		http://salimma.freeshell.org
GPG/PGP key:	http://salimma.freeshell.org/files/crypto/publickey.asc

__________________________________________________
Do You Yahoo!?
Everything you'll ever need on one web page
from News and Sport to Email and Music Charts
http://uk.my.yahoo.com

_______________________________________________
RPM-List mailing list <RPM-List@freshrpms.net>
http://lists.freshrpms.net/mailman/

In [21]:
print(email_body_to_words(sample_body, stopwords, stemmer))

['hello', 'anyon', 'made', 'work', 'sourc', 'rpm', 'dvd', 'rip', 'red', 'hat', 'number', 'number', 'matthia', 'spec', 'file', 'site', 'number', 'number', 'coupl', 'spec', 'file', 'lie', 'around', 'dvd', 'rip', 'websit', 'includ', 'one', 'patch', 'ago', 'appear', 'makefil', 'automat', 'generat', 'tri', 'instal', 'perl', 'librari', 'system', 'also', 'moment', 'dvd', 'rip', 'need', 'call', 'perlio', 'stdio', 'seem', 'work', 'perlio', 'perl', 'sure', 'cleanest', 'way', 'fix', 'anyon', 'work', 'thank', 'michã', 'l', 'alexandr', 'salim', 'web', 'httpaddress', 'gpg', 'pgp', 'key', 'httpaddress', 'yahoo', 'everyth', 'ever', 'need', 'one', 'web', 'page', 'news', 'sport', 'email', 'music', 'chart', 'httpaddress', 'rpm', 'list', 'mail', 'list', 'emailaddress', 'httpaddress']
